<a href="https://colab.research.google.com/github/tejashreereddyy/FMML-Project-and-Labs/blob/main/Copy_of_AIML_III_Module_01_Lab_03_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data Augmentation

Module 1, Lab 3

Data augmentation is a technique used to increase the diversity of your training data without collecting new data. By applying random transformations such as rotation and shear, we can make our model more robust and improve its generalization performance.

Rotation: This transformation rotates the image by a random angle within a specified range. For instance, setting the rotation range to 20 means the image will be rotated by a random angle between -20 and 20 degrees.

Shear: This transformation shears the image by a random factor within a specified range. Shearing means tilting the image in a way that some parts of it are pushed sideways, creating a sort of slanted look.

Questions

1. What is the best value for angle constraint and shear constraint you got? How much did the accuracy improve as compared to not using augmentations?

We need to perform a grid search over different values of rotation and shear to find the best combination.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.optimizers import Adam

# Load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess data
x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define a simple CNN model
def create_model():
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Define the range of values for rotation and shear
angle_range = np.arange(0, 91, 10)
shear_range = np.arange(0, 1.1, 0.1)

# Placeholder for results
results = []

# Grid search over the range of values
for angle in angle_range:
    for shear in shear_range:
        # Data augmentation with current parameters
        datagen = ImageDataGenerator(rotation_range=angle, shear_range=shear)
        datagen.fit(x_train)

        # Create and train the model
        model = create_model()
        history = model.fit(datagen.flow(x_train, y_train, batch_size=128),
                            validation_data=(x_test, y_test),
                            epochs=10, verbose=0)

        # Evaluate the model
        _, accuracy = model.evaluate(x_test, y_test, verbose=0)
        results.append((angle, shear, accuracy))
        print(f"Angle: {angle}, Shear: {shear}, Accuracy: {accuracy:.4f}")

# Find the best parameters
best_params = max(results, key=lambda x: x[2])
print(f"Best parameters - Angle: {best_params[0]}, Shear: {best_params[1]}, Accuracy: {best_params[2]:.4f}")


11490434/11490434 [==============================] - 0s 0us/step


2. Comparing with No Augmentation

Train a model without any augmentation and measure the accuracy.

In [ ]:
# Train without augmentation
model_no_aug = create_model()
history_no_aug = model_no_aug.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, verbose=0)
_, accuracy_no_aug = model_no_aug.evaluate(x_test, y_test, verbose=0)

print(f"Accuracy without augmentation: {accuracy_no_aug:.4f}")


3. Increasing Number of Augmentations per Sample

Evaluate the impact of increasing the number of augmentations.

In [ ]:
# Number of augmentations per sample
augmentations = [1, 2, 5, 10]
results_augmentations = []

for num_aug in augmentations:
    datagen = ImageDataGenerator(rotation_range=best_params[0], shear_range=best_params[1])
    datagen.fit(x_train)

    # Create augmented dataset
    augmented_data = []
    augmented_labels = []
    for _ in range(num_aug):
        for x, y in datagen.flow(x_train, y_train, batch_size=len(x_train)):
            augmented_data.append(x)
            augmented_labels.append(y)
            break
    augmented_data = np.vstack(augmented_data)
    augmented_labels = np.vstack(augmented_labels)

    # Train the model
    model_aug = create_model()
    history_aug = model_aug.fit(augmented_data, augmented_labels, validation_data=(x_test, y_test), epochs=10, verbose=0)
    _, accuracy_aug = model_aug.evaluate(x_test, y_test, verbose=0)

    results_augmentations.append((num_aug, accuracy_aug))
    print(f"Number of augmentations: {num_aug}, Accuracy: {accuracy_aug:.4f}")

# Plot the results
plt.plot(augmentations, [x[1] for x in results_augmentations])
plt.xlabel('Number of Augmentations')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Number of Augmentations')
plt.show()


4. Implementing Custom Augmentations

Implement a few custom augmentations and experiment.

In [ ]:
# Custom augmentations (example: horizontal flip, brightness adjustment)
datagen_custom = ImageDataGenerator(rotation_range=best_params[0],
                                    shear_range=best_params[1],
                                    horizontal_flip=True,
                                    brightness_range=[0.8, 1.2])

datagen_custom.fit(x_train)

# Train the model with custom augmentations
model_custom = create_model()
history_custom = model_custom.fit(datagen_custom.flow(x_train, y_train, batch_size=128),
                                  validation_data=(x_test, y_test),
                                  epochs=10, verbose=0)
_, accuracy_custom = model_custom.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy with custom augmentations: {accuracy_custom:.4f}")


5. Combining Various Augmentations

Combine different augmentations to find the highest accuracy.

In [ ]:
# Combined augmentations
datagen_combined = ImageDataGenerator(rotation_range=best_params[0],
                                      shear_range=best_params[1],
                                      horizontal_flip=True,
                                      zoom_range=0.2,
                                      brightness_range=[0.8, 1.2])

datagen_combined.fit(x_train)

# Train the model with combined augmentations
model_combined = create_model()
history_combined = model_combined.fit(datagen_combined.flow(x_train, y_train, batch_size=128),
                                      validation_data=(x_test, y_test),
                                      epochs=10, verbose=0)
_, accuracy_combined = model_combined.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy with combined augmentations: {accuracy_combined:.4f}")


6. Smallest Training Dataset for 50% Accuracy

Determine the smallest training dataset size that achieves accuracy above 50%.

In [ ]:
# Reduce training dataset sizes
dataset_sizes = [0.1, 0.25, 0.5, 0.75, 1.0]
results_dataset_sizes = []

for size in dataset_sizes:
    subset_x_train = x_train[:int(len(x_train) * size)]
    subset_y_train = y_train[:int(len(y_train) * size)]

    # Train the model
    model_subset = create_model()
    history_subset = model_subset.fit(subset_x_train, subset_y_train, validation_data=(x_test, y_test), epochs=10, verbose=0)
    _, accuracy_subset = model_subset.evaluate(x_test, y_test, verbose=0)

    results_dataset_sizes.append((size, accuracy_subset))
    print(f"Dataset size: {size * 100:.1f}%, Accuracy: {accuracy_subset:.4f}")

# Plot the results
plt.plot([size * 100 for size in dataset_sizes], [x[1] for x in results_dataset_sizes])
plt.xlabel('Training Dataset Size (%)')
plt.ylabel('Accuracy')
plt.title('Accuracy vs Training Dataset Size')
plt.show()


**Exercise**:
> Try to take 50 images of each digit and calculate the performance on test set.


In [ ]:
# Take 50 images of each digit
x_train_50 = []
y_train_50 = []

for digit in range(10):
    indices = np.where(y_train.argmax(axis=1) == digit)[0][:50]
    x_train_50.append(x_train[indices])
    y_train_50.append(y_train[indices])

x_train_50 = np.vstack(x_train_50)
y_train_50 = np.vstack(y_train_50)

# Data augmentation with best parameters
datagen_50 = ImageDataGenerator(rotation_range=best_params[0], shear_range=best_params[1])
datagen_50.fit(x_train_50)

# Train the model with 50 images per digit
model_50 = create_model()
history_50 = model_50.fit(datagen_50.flow(x_train_50, y_train_50, batch_size=128),
                          validation_data=(x_test, y_test),
                          epochs=10, verbose=0)
_, accuracy_50 = model_50.evaluate(x_test, y_test, verbose=0)
print(f"Accuracy with 50 images per digit: {accuracy_50:.4f}")


**Conclusion:**

Best values for angle and shear constraints were found using a grid search.

Accuracy improved compared to no augmentations.

Increasing the number of augmentations



